<a href="https://colab.research.google.com/github/Kaivan-Khazeni/Machine_Learning/blob/main/HW1_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import io
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

import numpy as np
import statsmodels.formula.api as smf
import seaborn as sb

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
labels_value = ['unacc','acc','good','vgood']
#label is where the values of each item is stored.  It is not binary,
#it is unacceptable, acceptable, good, very good.  This is different 
#then before
attributes = ['buying','maint','doors','persons','lug_boot','safety','label']

In [4]:
df = pd.read_csv ('train.csv',names=attributes)
df.head(5)

,buying,maint,doors,persons,lug_boot,safety,label
0,low,vhigh,4,4,big,med,acc
1,low,high,5more,4,med,high,vgood
2,vhigh,med,2,2,big,high,unacc
3,high,high,2,2,small,high,unacc
4,vhigh,low,3,2,big,low,unacc


In [5]:
test_dict = {'Outlook':['S','S','O','R','R','R','O','S','S','R','S','O','O','R'],
         'Temp':['H','H','H','M','C','C','C','M','C','M','M','M','H','M'],
         'Humid':['H','H','H','H','N','N','N','H','N','N','N','H','N','H'],
         'Wind':['W','S','W','W','W','S','S','W','W','W','S','S','W','S'],
         'Play':['-','-','+','+','+','-','+','-','+','+','+','+','+','-']}
test_df = pd.DataFrame(test_dict)
test_df

,Outlook,Temp,Humid,Wind,Play
0,S,H,H,W,-
1,S,H,H,S,-
2,O,H,H,W,+
3,R,M,H,W,+
4,R,C,N,W,+
5,R,C,N,S,-
6,O,C,N,S,+
7,S,M,H,W,-
8,S,C,N,W,+
9,R,M,N,W,+


In [66]:
dummy_df = df[["buying" ,"label"]]

dummy_df['buying'].unique()


array(['low', 'vhigh', 'high', 'med'], dtype=object)

In [96]:
dummy_low = dummy_df[df.buying =='med']
print(len(dummy_low))
len(dummy_low[dummy_low.label == "good"])

248


17

In [72]:
dummy_low.label.unique()

array(['acc', 'vgood', 'unacc', 'good'], dtype=object)

In [102]:
test_dictB = {'A':['K','A','I','K','A'],
         'B':['A','A','C','B','B'],
         'Y':['yes','no','so','maybe','so']}
test_dfB = pd.DataFrame(test_dictB)
test_dfB['A']

0    K
1    A
2    I
3    K
4    A
Name: A, dtype: object

In [7]:
attribute_values = {
    "buying": df['buying'].unique(),
    "maint": df['maint'].unique(),
    "doors": df['doors'].unique(),
    "persons": df['persons'].unique(),
    "lug_boot": df['lug_boot'].unique(),
    "safety": df['safety'].unique()
}

In [8]:
hey = test_df[['Outlook',test_df.keys()[-1]]]
hey[hey["Outlook"] == "S"]

,Outlook,Play
0,S,-
1,S,-
7,S,-
8,S,+
10,S,+


In [150]:
decision_table = ID3(6,2,test_df,"none",tree=None)
decision_table

{'Humid': 0.07142857142857145,
 'Outlook': 0.0714285714285714,
 'Temp': 5.551115123125783e-17,
 'Wind': 5.551115123125783e-17}

In [138]:
import pprint

pprint.pprint(decision_table)

{'Humid': 0.0, 'Outlook': 0.0, 'Temp': 0.0, 'Wind': 0.0}


ID3 function below will take in a depth parameter and a variant indicator.  This way, we can specify if we want
to use Entropy, ME, or Gini Index.  It will also take in the S table.

In [140]:
#max depth will range from 1 - 6
def ID3(max_depth, variant,S,A,tree=None):
  Class = S.keys()[-1]
  #Step 1: check if all examplesa have the same label
  #keep counter to make sure iterations do not exceed the max depth
  if max_depth >= 0:
    
    #print("In this call 1, 2 ..")
    #Finding root based off of gain
    #A. Find Total entropy using variant
    total_entropy = find_total_entropy(variant,S)
    #print("Found total entropy 1,2")
    #print(total_entropy)
    #B. Now find the best split, which is A.
    table_attributes = S.loc[:,S.columns != S.keys()[-1]].columns
    #print(table_attributes)

    best_A = find_best_split(variant,table_attributes,S,total_entropy)
    #print("Found A")
   
    return best_A
    A = sorted(best_A.items(), key=lambda x: x[1],reverse=True)[0][0]

    #print(A)
    if tree is None:                    
      tree={}
      tree[A] = {}

    
    values_of_A = S[A].unique()

    for A_val in values_of_A:
      Sv = S.loc[S[A]==A_val,S.columns != A]
      common,counts = np.unique(Sv[Sv.keys()[-1]],return_counts=True)

      if len(counts) == 1:
        tree[A][A_val] = common[0]
      else:
        tree[A][A_val] = ID3(max_depth-1,variant,Sv,A,tree=None)     
    return tree
    


      


In [149]:
def find_best_split(variant,attr,S,total_ent):
  #Need to check the
  
  list_of_IG = {}
  for A1 in attr:
    curr_attr = A1
    curr_df = S[[curr_attr,S.keys()[-1]]]
    
    len_A1_df = len(curr_df[curr_attr])
    found_sum_expected_entropy = 0
    all_attr_values = curr_df[curr_attr].unique()
    #print(all_attr_values)
    #print(counter)
    for A2 in all_attr_values:
      
      attribute_specific = curr_df[curr_df[curr_attr] == A2]
      #print(attribute_specific)
      found_expected_entropy = 0
      entropy = find_total_entropy(variant,attribute_specific)
      if (entropy == 1 and variant == 2):
        entropy = 0
      #print(entropy)
      found_sum_expected_entropy += (entropy * (len(attribute_specific)/len_A1_df))
    #print(found_sum_expected_entropy)
    list_of_IG[A1] = total_ent - found_sum_expected_entropy   
  return list_of_IG


In [12]:
def find_gain(total_entropy,expec):
  if variant == 1:
    #use entropy
    d
  if variant == 2:
    d
    #use ME
  if variant == 3:
    d
    #use Gini Index

In [148]:
def find_total_entropy(variant,S):
  if variant == 1:
    #use entropy
    type_name = S.keys()[-1]
    entropy = 0
    label_names = S[type_name].unique()
    for val in label_names:
      #print(val)
      p = S[type_name].value_counts()[val] / len(S[type_name])
      #print(p)
      entropy += -p*np.log2(p)
    return entropy
  elif variant == 2:
    counter = len(labels_value)
    
    #use ME
    type_name = S.keys()[-1]  
    #ME = 0
    label_names = S[type_name].unique()
    list_of_prob = []
    for val in label_names:
      #print(S)
      #print(val)
      #print(S[type_name].value_counts()[val])
      p = S[type_name].value_counts()[val] / len(S)
      list_of_prob.append(p)
    return np.min(list_of_prob)
  elif variant == 3:
    #use gini
    type_name = S.keys()[-1]
    #Gini = 0
    label_names = S[type_name].unique()
    sum_p = 0
    for val in label_names:
      p = S[type_name].value_counts()[val] / len(S[type_name])
      sum_p += np.square(p)

    return 1 - sum_p